<a href="https://colab.research.google.com/github/SumitBr/SumitBr/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

Video tutorial available here: https://www.youtube.com/watch?v=G4Q06-sEEQs

With this notebook you can use stable diffusion to generate images. 

You need a hugging face account to use this notebook. You can sign up here: https://huggingface.co/join

If you don't have a credential, follow the link in code snippet 2 to get one! Also make sure you follow the rules outlined in the License for the model, which can be found here: https://huggingface.co/spaces/CompVis/stable-diffusion-license

Make sure you read the whole license including the use restrictions which are as follows:

Use Restrictions 
 
You agree not to use the Model or Derivatives of the Model: 
- In any way that violates any applicable national, federal, state, local 
or international law or regulation; 
- For the purpose of exploiting, harming or attempting to exploit or harm 
minors in any way; 
- To generate or disseminate verifiably false information and/or content 
with the purpose of harming others; 
- To generate or disseminate personal identifiable information that can 
be used to harm an individual; 
- To defame, disparage or otherwise harass others; 
- For fully automated decision making that adversely impacts an 
individual’s legal rights or otherwise creates or modifies a binding, 
enforceable obligation; 
- For any use intended to or which has the effect of discriminating 
against or harming individuals or groups based on online or offline 
social behavior or known or predicted personal or personality 
characteristics; 
- To exploit any of the vulnerabilities of a specific group of persons 
based on their age, social, physical or mental characteristics, in order 
to materially distort the behavior of a person pertaining to that group 
in a manner that causes or is likely to cause that person or another 
person physical or psychological harm; 
- For any use intended to or which has the effect of discriminating 
against individuals or groups based on legally protected characteristics 
or categories; 
- To provide medical advice and medical results interpretation; 
- To generate or disseminate information for the purpose to be used for 
administration of justice, law enforcement, immigration or asylum 
processes, such as predicting an individual will commit fraud/crime 
commitment (e.g. by text profiling, drawing causal relationships between 
assertions made in documents, indiscriminate and arbitrarily-targeted 
use).

I built this notebook using code from Woctezuma, check out their github here: https://github.com/woctezuma

In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate

In this block of code it will ask for your token from Hugging Face. 

If you don't have an account you can make one here: https://huggingface.co/join

Before you can do this you need to accept the license agreement for the model, which you can do here: https://huggingface.co/CompVis/stable-diffusion-v1-4

On the token page select "new token" and ask for a "read" token (you can name it "Colab").

In [ ]:
!huggingface-cli login

Now we set up our model and pipeline. IF you want to remove the NSFW safety checker, change "remove_safety" to "True".

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler

scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)

In [ ]:
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"
remove_safety = False


pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
if remove_safety:
  pipe.safety_checker = lambda images, clip_input: (images, False)
pipe = pipe.to(device)



Here is where you actually make images. Change the "prompt" to whatever you want to try and then change "num_images" if you want more than one image generated. You can re-run this cell without having to re-run everything before it, just FYI.

In [ ]:
prompt = "photo of human hands"
num_images = 2

prompts = [ prompt ] * num_images
with autocast("cuda"):
    images = pipe(prompts, guidance_scale=7.5, num_inference_steps=50).images
    
media.show_images(images)
images[0].save("output.jpg")

In [ ]:
!ls